In [1]:
import sys
import os

sys.path.insert(0, os.path.abspath("."))
sys.path.append(os.path.abspath("../"))

In [2]:
import numpy as np
np.set_printoptions(linewidth=np.inf)
import matplotlib.pyplot as plt
%matplotlib inline
import plotly.graph_objects as go
import functools
from scipy.linalg import qr_insert, qr

In [3]:
import desc

from desc.basis import *
from desc.backend import *
from desc.compute import data_index
from desc.coils import SplineXYZCoil
from desc.equilibrium import *
from desc.examples import get
from desc.grid import *
from desc.geometry import *

from desc.objectives import *
from desc.objectives.getters import *
from desc.objectives.normalization import compute_scaling_factors
from desc.objectives.utils import factorize_linear_constraints
from desc.optimize._constraint_wrappers import *

from desc.transform import Transform
from desc.plotting import *
from desc.optimize import Optimizer
from desc.perturbations import *
from desc.profiles import PowerSeriesProfile

from desc.utils import copy_coeffs, svd_inv_null

from desc.__main__ import main
from desc.vmec_utils import vmec_boundary_subspace
from desc.input_reader import InputReader
from desc.continuation import solve_continuation_automatic

DESC version 0.12.1+1063.g6ab8327ee,using JAX backend, jax version=0.4.31, jaxlib version=0.4.31, dtype=float64
Using device: CPU, with 9.42 GB available memory


# POINCARE

In [11]:
eq = get("precise_QA")
constraints = get_fixed_boundary_constraints(eq)
constraints = maybe_add_self_consistency(eq, constraints)
constraint = ObjectiveFunction(constraints)
objective = ObjectiveFunction(ForceBalance(eq))
objective = LinearConstraintProjection(objective, constraint)

objective.build()

x0 = objective.x(eq)
print(objective.jac_scaled_error(x0).shape)
print(objective._A.shape)

Building objective: force
Precomputing transforms
Building objective: lcfs R
Building objective: lcfs Z
Building objective: fixed Psi
Building objective: fixed pressure
Building objective: fixed current
Building objective: fixed sheet current
Building objective: self_consistency boundary R
Building objective: self_consistency boundary Z
Building objective: lambda gauge
Building objective: self_consistency axis R
Building objective: self_consistency axis Z
Building objective: self_consistency section R
Building objective: self_consistency section Z
Building objective: self_consistency section Lambda
(5346, 856)
(299, 1155)


In [12]:
print(objective._Z.shape)
print(objective.dim_f)
print(objective.dim_x)
print(f"{eq.R_lmn.shape[0] + eq.Z_lmn.shape[0] + eq.L_lmn.shape[0]}")

(1155, 856)
5346
1527
1145


In [7]:
sum = 0
for key in eq.params_dict.keys():
    value = eq.params_dict[key]
    if value.shape[0] != 0:
        print(f"{key:<10} {value.shape[0]}")
    sum += value.shape[0]
print(sum)

R_lmn      385
Z_lmn      380


L_lmn      380
p_l        5
c_l        5
Psi        1
Rp_lmn     25
Zp_lmn     20
Lp_lmn     20
Ra_n       9
Za_n       8
Rb_lmn     145
Zb_lmn     144
1527


In [8]:
eq00 = get("precise_QA")
constraints = get_fixed_boundary_constraints(eq00)
objective = ObjectiveFunction(ForceBalance(eq00))
eq00.solve(constraints=constraints, objective=objective, maxiter=0)

Building objective: force
Precomputing transforms
Building objective: lcfs R
Building objective: lcfs Z
Building objective: fixed Psi
Building objective: fixed pressure
Building objective: fixed current
Building objective: fixed sheet current
Building objective: self_consistency boundary R
Building objective: self_consistency boundary Z
Building objective: lambda gauge
Building objective: self_consistency axis R
Building objective: self_consistency axis Z
Building objective: self_consistency section R
Building objective: self_consistency section Z
Building objective: self_consistency section Lambda
Number of parameters: 856
Number of objectives: 5346

Starting optimization
Using method: lsq-exact
         Current function value: 4.398e-10
         Total delta_x: 0.000e+00
         Iterations: 0
         Function evaluations: 1
         Jacobian evaluations: 1
                                                                 Start  -->   End
Total (sum of squares):                       

(Equilibrium at 0x751dc184c950 (L=8, M=8, N=8, NFP=2, sym=True, spectral_indexing=ansi),
     message: Maximum number of iterations has been exceeded.
     success: False
         fun: [ 3.778e-08  4.955e-08 ...  2.533e-08  4.359e-07]
           x: [ 9.308e-05  1.808e-04 ... -2.110e-02 -1.468e-01]
         nit: 0
        cost: 4.397585731164801e-10
           v: [ 1.000e+00  1.000e+00 ...  1.000e+00  1.000e+00]
  optimality: 0.000385648537564147
        nfev: 1
        njev: 1
        allx: [Array([ 1.201e-08,  2.337e-07, ...,  1.857e-09,  2.063e-10],      dtype=float64)]
       alltr: [Array( 1.512e+02, dtype=float64)]
     history: [[{'R_lmn': Array([ 1.201e-08,  2.337e-07, ..., -3.944e-08, -1.150e-09],      dtype=float64), 'Z_lmn': Array([-6.749e-08,  4.848e-08, ...,  9.626e-08,  5.448e-08],      dtype=float64), 'L_lmn': Array([-5.726e-08, -3.735e-07, ...,  5.368e-07, -1.214e-07],      dtype=float64), 'p_l': Array([ 0.000e+00,  0.000e+00,  0.000e+00,  0.000e+00,
                    

# MASTER

In [25]:
eq = get("precise_QA")
constraints = get_fixed_boundary_constraints(eq)
constraints = maybe_add_self_consistency(eq, constraints)
constraint = ObjectiveFunction(constraints)
objective = ObjectiveFunction(ForceBalance(eq))
objective = LinearConstraintProjection(objective, constraint)

objective.build()

x0 = objective.x(eq)
x0_full = objective.recover(x0)
Z = objective._Z
xh = Z @ x0
# print(x0.shape)
# print(xh.shape)
print(objective._xp.shape)
print(objective._unfixed_idx.shape)
# print(x0_full.shape)
# print(objective.jac_scaled_error(x0).shape)
print(objective._A.shape)

Building objective: force
Precomputing transforms
Building objective: lcfs R
Building objective: lcfs Z
Building objective: fixed Psi
Building objective: fixed pressure
Building objective: fixed current
Building objective: fixed sheet current
Building objective: self_consistency R
Building objective: self_consistency Z
Building objective: lambda gauge
Building objective: axis R self consistency
Building objective: axis Z self consistency
(1462,)
(1094,)
(238, 1094)


In [26]:
print(objective._Z.shape)
print(objective.dim_f)
print(objective.dim_x)
print(f"{eq.R_lmn.shape[0] + eq.Z_lmn.shape[0] + eq.L_lmn.shape[0]}")
print(f"{eq.Rb_lmn.shape[0] + eq.Zb_lmn.shape[0]}")

(1094, 856)
5346
1462
1145
289


In [17]:
1145-289

856

In [28]:
1492-289

1203

In [7]:
sum = 0
for key in eq.params_dict.keys():
    value = eq.params_dict[key]
    if value.shape[0] != 0:
        print(f"{key:<10} {value.shape[0]}")
    sum += value.shape[0]
print(sum)

R_lmn      385
Z_lmn      380
L_lmn      380
p_l        5
c_l        5
Psi        1
Ra_n       9
Za_n       8
Rb_lmn     145
Zb_lmn     144
1462


In [6]:
eq00 = get("precise_QA")
constraints = get_fixed_boundary_constraints(eq00)
objective = ObjectiveFunction(ForceBalance(eq00))
eq00.solve(constraints=constraints, objective=objective, maxiter=0)

Building objective: force
Precomputing transforms
Building objective: lcfs R
Building objective: lcfs Z
Building objective: fixed Psi
Building objective: fixed pressure
Building objective: fixed current
Building objective: fixed sheet current
Building objective: self_consistency R
Building objective: self_consistency Z
Building objective: lambda gauge
Building objective: axis R self consistency
Building objective: axis Z self consistency
Number of parameters: 856
Number of objectives: 5346

Starting optimization
Using method: lsq-exact
         Current function value: 4.398e-10
         Total delta_x: 0.000e+00
         Iterations: 0
         Function evaluations: 1
         Jacobian evaluations: 1
                                                                 Start  -->   End
Total (sum of squares):                                      4.398e-10  -->   4.398e-10, 
Maximum absolute Force error:                                1.962e+01  -->   1.962e+01 (N)
Minimum absolute Force error

(Equilibrium at 0x74efab1446d0 (L=8, M=8, N=8, NFP=2, sym=True, spectral_indexing=ansi),
     message: Maximum number of iterations has been exceeded.
     success: False
         fun: [ 3.778e-08  4.955e-08 ...  2.533e-08  4.359e-07]
           x: [-8.456e-05  5.634e-05 ... -2.029e-02 -1.468e-01]
         nit: 0
        cost: 4.397585730520889e-10
           v: [ 1.000e+00  1.000e+00 ...  1.000e+00  1.000e+00]
  optimality: 0.000561200959718055
        nfev: 1
        njev: 1
        allx: [Array([ 1.201e-08,  2.337e-07, ...,  1.857e-09,  2.063e-10],      dtype=float64)]
       alltr: [Array( 1.363e+02, dtype=float64)]
     history: [[{'R_lmn': Array([ 1.201e-08,  2.337e-07, ..., -3.944e-08, -1.150e-09],      dtype=float64), 'Z_lmn': Array([-6.749e-08,  4.848e-08, ...,  9.626e-08,  5.448e-08],      dtype=float64), 'L_lmn': Array([-5.726e-08, -3.735e-07, ...,  5.368e-07, -1.214e-07],      dtype=float64), 'p_l': Array([ 0.000e+00,  0.000e+00,  0.000e+00,  0.000e+00,
                    